In [16]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import utils
import constants
import chess
import chess.pgn
import numpy as np
import os
import random

Data files: 1 - 27
 - each has 1 000 000 rows
 - treat 1 - 25 as training data
 - 26 as final test data
 - 27 as validation data after training is completed (for preliminary comparisons)

In [17]:
def split_pgn():
    pgn_files = [f"data/chessgames20{i}.pgn" for i in range(16, 22)]
    outcomes = {"1":1, "1/2":0.5, "0":0}
    df = []
    file_counter = 1
    for pgn_file in pgn_files:
        with open(pgn_file, "r") as f:
            game = chess.pgn.read_game(f)
            while game != None:
                result = game.headers["Result"].split("-")
                result = outcomes[result[0]]
                board = game.board()
                for move in game.mainline_moves():
                    df.append({"board":board.fen(), "move":move.uci(), "outcome":result})
                    board.push(move)
                if len(df) >= 1000000:
                    df = pd.DataFrame(df)
                    df.to_csv(f"data/split/chess{file_counter}.csv", index=False)
                    file_counter += 1
                    df = []
                game = chess.pgn.read_game(f)

def process(file_numbers):
    for n in file_numbers:
        df = pd.read_csv(f"data/split/chess{n}.csv")
        df["obs"] = df["board"].map(utils.parse_fen)
        df.to_pickle(f"data/split/processed/chess{n}.pkl")
# split_pgn()
process([1, 2, 3, 4, 5])

In [3]:
def restrict_legal(move_arr, fen):
    ''' Restricts the move probability array to only legal moves and rescale remaining legal move probabilities '''
    board = chess.Board(fen)
    legal_arr = np.zeros(constants.LEN_UCI_MOVES, dtype=np.int16)
    for move in board.legal_moves:
        legal_arr[constants.uci_moves[move.uci()]] = 1
    for i in range(constants.LEN_UCI_MOVES):
        if legal_arr[i] == 0:
            move_arr[i] = 0
    return move_arr / np.sum(move_arr)

In [28]:
def train_model(model, data_dir="data/split/", n_files=25, epochs=1, ckpt_dir="ckpt/"):

    file_order = list(range(1, n_files+1))

    if not os.path.exists(ckpt_dir):
        os.mkdir(ckpt_dir)
    if not os.path.exists(utils.path_join(ckpt_dir, model.name)):
        os.mkdir(utils.path_join(ckpt_dir, model.name))

    checkpoints = [utils.path_join(ckpt_dir, model.name, name) for name in os.listdir(utils.path_join(ckpt_dir, model.name))]
    if checkpoints:
        latest_checkpoint = max(checkpoints, key=os.path.getctime)
        print("Restoring from", latest_checkpoint)
        model = keras.models.load_model(latest_checkpoint)
        n = latest_checkpoint.split("-")[-1][0]
    else:
        n = 0

    processed_files = [filename.split(".")[0] for filename in os.listdir(utils.path_join(data_dir, "processed"))]
    for file in file_order:
        if file <= int(n):
            continue
        filename = f"chess{file}"
        print(f"Training on file {filename}")

        # load data from .csv file
        if filename in processed_files:
            df = pd.read_pickle(utils.path_join(data_dir, "processed", filename + ".pkl"))
        else:
            df = pd.read_csv(utils.path_join(data_dir, filename + ".csv"))

            # translate from fen to obs arr
            print("Processing...", end="")
            df.loc[:, "obs"] = df.loc[:, "board"].map(utils.parse_fen)
            print("complete")

        # convert to numpy arrays
        x = np.array(df["obs"].values.tolist())
        y = df["move"].values

        # train
        model.fit(x, y, batch_size=64, epochs=epochs, validation_split=0.1)

        # save checkpoint between files
        model.save(utils.path_join(ckpt_dir, model.name, f"{model.name}-{file}.h5"))
    print(f"Training completed. Final save file: {utils.path_join(ckpt_dir, model.name, f'{model.name}-{file}.h5')}")

In [29]:
def evaluate_model(model_names, data="validation", data_dir="data/split/", ckpt_dir="ckpt/", metric="sparse_categorical_accuracy"):

    models = []
    for model_name in model_names:
        checkpoints = [utils.path_join(ckpt_dir, model_name, name) for name in os.listdir(utils.path_join(ckpt_dir, model_name))]
        if checkpoints:
            latest_checkpoint = max(checkpoints, key=os.path.getctime)
            print("Restoring from", latest_checkpoint)
            model = keras.models.load_model(latest_checkpoint)
            models.append(model)
        else:
            print("No checkpoint available")

    # load data
    if data == "validation":
        data = pd.read_csv(utils.path_join(data_dir, "chess27.csv"))
    elif data == "test":
        data = pd.read_csv(utils.path_join(data_dir, "chess26.csv"))
    
    # convert fen to obs
    print("Processing...", end="")
    data.loc[:, "obs"] = data.loc[:, "board"].map(utils.parse_fen)
    print("complete")

    # convert to numpy arrays
    x = np.array(data["obs"].values.tolist())
    y = data["move"].values

    performance = []
    for model_name, model in zip(model_names, models):
        performance.append([model_name, model.evaluate(x, y, batch_size=128, return_dict=True)[metric]])

    return pd.DataFrame(performance, columns=["model_name", metric])

In [ ]:
from tf_models import conv2d_seq
model = conv2d_seq.co256_cl5_d0()
train_model(model, n_files=5)

In [45]:
models = ["conv2d_seq_cx_dr", "conv2d_seq_cx_dr2", "conv2d_seq_cxx_dr2", "conv2d_seq_c4_d0", "conv2d_seq_c5_d0"]
results = evaluate_model(models)
results

Restoring from ckpt/conv2d_seq_cx_dr/conv2d_seq_cx_dr-5.h5
Restoring from ckpt/conv2d_seq_cx_dr2/conv2d_seq_cx_dr2-5.h5
Restoring from ckpt/conv2d_seq_cxx_dr2/conv2d_seq_cxx_dr2-5.h5
Restoring from ckpt/conv2d_seq_c4_d0/conv2d_seq_c4_d0-5.h5
Restoring from ckpt/conv2d_seq_c5_d0/conv2d_seq_c5_d0-5.h5
Processing...complete
5451/5451 [==============================] - 69s 13ms/step - loss: 3.1406 - sparse_categorical_accuracy: 0.2478


,model_name,sparse_categorical_accuracy
0,conv2d_seq_cx_dr,0.236640
1,conv2d_seq_cx_dr2,0.240458
2,conv2d_seq_cxx_dr2,0.244143
3,conv2d_seq_c4_d0,0.244680
4,conv2d_seq_c5_d0,0.247849


In [7]:
import chess.pgn

file = "data/test.pgn"
with open(file, "r") as f:
    game = chess.pgn.read_game(f)
board = game.board()
for move in game.mainline_moves():
    board.push(move)
# print(board.outcome())
game.__dict__

{'parent': None,
 'move': None,
 'variations': [<ChildNode at 0x11840d420 (1. e4 ...)>],
 'comment': '',
 'starting_comment': '',
 'nags': set(),
 'headers': Headers(Event='FICS rated standard game', Site='FICS freechess.org', Date='2016.12.31', Round='?', White='Ashokag', Black='MortimerBlackwell', Result='0-1', BlackClock='0:20:00.000', BlackElo='2416', BlackIsComp='Yes', BlackRD='na', ECO='B10', FICSGamesDBGameNo='410004285', PlyCount='43', Time='23:21:00', TimeControl='1200+10', WhiteClock='0:20:00.000', WhiteElo='1593', WhiteRD='na'),
 'errors': []}

In [18]:
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv("distribution.csv")
df.head()

,turn,state,action,visits,reward,ucb
0,0,rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w ...,g1h3,375,172.600000,0.769440
1,0,rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w ...,g1f3,404,190.666667,0.769817
2,0,rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w ...,b1c3,399,187.516667,0.769697
3,0,rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w ...,b1a3,375,172.666667,0.769617
4,0,rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w ...,h2h3,393,183.733333,0.769524


In [ ]:
for turn, d in df.groupby(["turn"]):
    print(turn)
    ds = d.sort_values("visits", ascending=False)
    plt.plot("action", "visits", data=ds)
    plt.show()